<a href="https://colab.research.google.com/github/BroccoliWarrior/transformer-basic-knowledge/blob/main/Decoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***解码技术（Decoding）***

  在自然语言处理（NLP）的生成任务中，如机器翻译、文本摘要、对话生成等，模型在接收到输入后，需要通过**解码技术**生成符合语法和语义的输出序列。解码技术的核心是如何从模型预测的概率分布中选择下一个token（词或子词），最终生成完整的序列


1. **解码技术的基本概念**

在序列生成任务中，模型（如Transformer、RNN等）通常采用**自回归（Auto-regressive）** 方式生成文本：
  - 给定已生成的序列前缀 $x_1, x_2, ..., x_t$，模型预测下一个token $x_{t+1}$ 的概率分布 $P(x_{t+1} | x_1, ..., x_t)$。
  - 解码技术就是根据这个概率分布选择 $x_{t+1}$ 的策略。

  模型输出的原始得分（logits）会通过 **Softmax函数** 转换为概率分布：

  $$p_i = \frac{e^{z_i}}{\\sum_{j=1}^{|V|} e^{z_j}}$$

  其中，$z_i$ 是第 $i$ 个token的原始得分，$|V|$ 是词汇表大小，$p_i$ 是第 $i$ 个token被选中的概率

2. **常见的解码技术**

2.1 *贪心解码（Greedy Decoding）*

  **核心思想**：每一步都选择概率最高的token作为下一个token。

  $$x_{t+1} = \arg\max_{x}P(x | x_1, ..., x_t)$$

  **示例**：
  - 假设当前前缀是 \"I like\"，模型预测下一个token的概率分布为：\"cats\"(0.6)、\"dogs\"(0.3)、\"books\"(0.1)，则贪心解码会选择 \"cats\"

  **优点**：
  - 计算速度快，无需复杂计算。

  **缺点**：
  - 短视性：局部最优不一定是全局最优。例如，某一步选择高概率token可能导致后续token概率极低，最终生成的序列整体质量差。
  - 多样性差：生成的序列往往缺乏变化。"

2.2 *束搜索（Beam Search）*

  **核心思想**：每一步保留概率最高的 $k$ 个候选序列（束宽为 $k$），最终从这些候选中选择最优序列

  **步骤**：
  1. 初始时，候选序列为起始符 `<s>`
  2. 对每个候选序列，扩展所有可能的下一个token，计算新序列的概率（通常用对数概率之和，避免数值下溢）
  3. 保留概率最高的 $k$ 个新序列，重复步骤2，直到生成结束符 `</s>` 或达到最大长度

  **示例**：$k=2$
  - 第一步：候选序列为 `<s>`，扩展后选择概率最高的2个token，如 \"I\"(0.8) 和 \"You\"(0.7)
  - 第二步：对 \"I\" 和 \"You\" 分别扩展，保留概率最高的2个序列，如 \"I like\"(0.8×0.6=0.48) 和 \"You love\"(0.7×0.7=0.49)

  **优点**：
  - 相比贪心解码，能找到更优的全局序列，生成质量更高（尤其在机器翻译等任务中，能提升BLEU分数）

  **缺点**：
  - 计算复杂度随束宽 $k$ 增加而提高
  - 仍偏向保守，生成的序列多样性不足

2.3 *随机采样（Random Sampling）*

  **核心思想**：根据模型预测的概率分布随机选择下一个token，概率高的token被选中的可能性大，但低概率token也有机会被选中

  **示例**：
  - 概率分布为 \"cats\"(0.6)、\"dogs\"(0.3)、\"books\"(0.1) 时，有60%概率选 \"cats\"，30%选 \"dogs\"，10%选 \"books\"

  **优点**：
  - 生成的序列具有多样性，适合创意写作、故事生成等需要变化的任务

  **缺点**：
  - 可能生成不合理或无意义的序列（尤其是低概率token被选中时）

2.4 *top-k 采样（Top-k Sampling）*

  **核心思想**：只从概率最高的 $k$ 个token中随机采样，过滤掉低概率token，减少无意义序列的生成概率

  **步骤**：
  1. 选择概率最高的 $k$ 个token（如 $k=50$）
  2. 重新归一化这 $k$ 个token的概率（确保和为1）
  3. 从这 $k$ 个token中随机采样

  **优点**：
  - 相比纯随机采样，生成的序列更合理，同时保留一定多样性,

  **缺点**：
  - $k$ 的选择较敏感，不同任务可能需要不同的 $k$ 值
  - 分布尖锐时不适合

2.5 *Nucleus 采样（Top-p Sampling）*

  **核心思想**：选择累积概率之和达到 $p$（通常 $p=0.9$）的最小token集合，从该集合中随机采样

  **步骤**：
  1. 将token按概率从高到低排序,
  2. 累加概率，直到和超过 $p$，此时包含的token组成候选集
  3. 重新归一化候选集的概率，随机采样

  **优点**：
  - 候选集大小随概率分布动态调整，避免了top-k中 $k$ 固定的问题
  - 生成质量和多样性较平衡，是目前大语言模型中常用的采样策略。"

2.6 *带温度的采样（Temperature Sampling）*

  **核心思想**：通过温度参数 $T$ 调节概率分布的“陡峭程度”，控制采样的随机性

  修改后的概率分布为：

  $$p_i = \frac{e^{z_i / T}}{\\sum_{j=1}^{|V|} e^{z_j / T}}$$

  - 当 $T=1$ 时，与普通随机采样一致
  - 当 $T < 1$ 时（如 $T=0.5$），概率分布更陡峭，高概率token被选中的概率更高，生成更确定
  - 当 $T > 1$ 时（如 $T=2$），概率分布更平缓，低概率token被选中的概率增加，生成更多样但可能更混乱

  **应用**：通过调整 $T$ 平衡生成的确定性和多样性

### 组合策略与实用技巧
#### 1. 常见组合
- **Temperature + Top-k**：先用温度缩放概率分布，再从概率最高的前$k$个Token中采样。
- **Temperature + Top-p**：先通过温度缩放调整分布，再选取概率质量之和≥$p$的Token集合进行采样。
- **Temperature + Beam Search**：在Beam Search扩展候选序列时，借助温度引入一定噪声，提升生成结果的多样性。

#### 2. 其他实用技巧
在实际应用中，还会结合以下额外技巧：
- **重复惩罚（repetition_penalty）**：对近期已生成过的Token，人为降低其被选中的概率，防止生成内容陷入重复循环。
- **长度惩罚（length_penalty）**：在机器翻译、文本摘要等任务中，用于鼓励生成更长或更短的句子，满足不同场景对文本长度的需求。
- **bad words ban**：禁止生成某些特定的关键词，可用于过滤不当内容或引导生成方向。